## Xavier Deep network version

In [1]:
import tensorflow as tf
import numpy as np
from time import time
import os

print(tf.__version__)

2022-05-02 14:17:20.616468: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


2.3.0


### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split("-")[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1)  # [N, 28, 28] -> [N, 28, 28, 1]
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = tf.keras.utils.to_categorical(train_labels, 10)  # [N, ] -> [N, 10]
    test_labels = tf.keras.utils.to_categorical(test_labels, 10)  # [N, ] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    return train_data, test_data

### Performance function

In [4]:
def loss_function(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits,
                                                                  y_true=labels,
                                                                  from_logits=True))
    return loss

def accuracy_function(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_function(model, images, labels)
    return tape.gradient(loss, model.variables)

### Create model (class version)

In [10]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform()
        
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Flatten())
        
        for i in range(4):
            self.model.add(tf.keras.layers.Dense(units=512, use_bias=True, kernel_initializer=weight_init))
            self.model.add(tf.keras.layers.Activation(tf.keras.activations.relu))
        
        self.model.add(tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init))
        
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### Define data & hyper-parameter

In [6]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
        shuffle(buffer_size=100000).\
        prefetch(buffer_size=batch_size).\
        batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
        shuffle(buffer_size=100000).\
        prefetch(buffer_size=len(test_x)).\
        batch(len(test_x))

2022-05-02 14:40:17.425776: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-05-02 14:40:17.511999: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:03:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-05-02 14:40:17.512599: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 0000:04:00.0 name: GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.79GiB deviceMemoryBandwidth: 417.23GiB/s
2022-05-02 14:40:17.512619: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-05-02 14:40:17.514157: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-05

### Define model & optimizer & writer

In [11]:
""" Model """
network = create_model_class(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """""
checkpoint_dir = "checkpoints"
logs_dir = "logs"

model_dir = "nn_deep"

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [13]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))
                
                train_loss = loss_function(network, train_input, train_label)
                train_accuracy = accuracy_function(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_function(network, test_input, test_label)
                
                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + "-{}".format(counter))

# test phase
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_function(network, test_input, test,label)
    
    print("test_Accuracy: %.4f" % test_accuracy)

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.8044, train_loss: 1.94889629, train_accuracy: 0.5859, test_Accuracy: 0.2550
Epoch: [ 0] [    1/  468] time: 1.0455, train_loss: 1.92502761, train_accuracy: 0.3984, test_Accuracy: 0.2691
Epoch: [ 0] [    2/  468] time: 1.3010, train_loss: 1.77621114, train_accuracy: 0.4688, test_Accuracy: 0.3674
Epoch: [ 0] [    3/  468] time: 1.5334, train_loss: 1.51144493, train_accuracy: 0.5938, test_Accuracy: 0.5271
Epoch: [ 0] [    4/  468] time: 1.7890, train_loss: 1.29824626, train_accuracy: 0.6562, test_Accuracy: 0.6040
Epoch: [ 0] [    5/  468] time: 2.0155, train_loss: 1.14241779, train_accuracy: 0.7031, test_Accuracy: 0.6824
Epoch: [ 0] [    6/  468] time: 2.2522, train_loss: 0.87561333, train_accuracy: 0.7344, test_Accuracy: 0.6854
Epoch: [ 0] [    7/  468] time: 2.4830, train_loss: 0.87610364, train_accuracy: 0.7578, test_Accuracy: 0.7563
Epoch: [ 0] [    8/  468] time: 2.7413,

Epoch: [ 0] [   74/  468] time: 18.6735, train_loss: 0.24561079, train_accuracy: 0.9062, test_Accuracy: 0.9300
Epoch: [ 0] [   75/  468] time: 18.9267, train_loss: 0.23157522, train_accuracy: 0.9375, test_Accuracy: 0.9296
Epoch: [ 0] [   76/  468] time: 19.1713, train_loss: 0.27405459, train_accuracy: 0.9375, test_Accuracy: 0.9223
Epoch: [ 0] [   77/  468] time: 19.4112, train_loss: 0.21009253, train_accuracy: 0.9219, test_Accuracy: 0.9183
Epoch: [ 0] [   78/  468] time: 19.6519, train_loss: 0.26862672, train_accuracy: 0.9219, test_Accuracy: 0.9283
Epoch: [ 0] [   79/  468] time: 19.8836, train_loss: 0.16072047, train_accuracy: 0.9375, test_Accuracy: 0.9335
Epoch: [ 0] [   80/  468] time: 20.1313, train_loss: 0.16981107, train_accuracy: 0.9375, test_Accuracy: 0.9340
Epoch: [ 0] [   81/  468] time: 20.3636, train_loss: 0.17280267, train_accuracy: 0.9453, test_Accuracy: 0.9314
Epoch: [ 0] [   82/  468] time: 20.6072, train_loss: 0.11322945, train_accuracy: 0.9688, test_Accuracy: 0.9293
E

Epoch: [ 0] [  148/  468] time: 36.2302, train_loss: 0.17116210, train_accuracy: 0.9375, test_Accuracy: 0.9509
Epoch: [ 0] [  149/  468] time: 36.4707, train_loss: 0.08700076, train_accuracy: 0.9766, test_Accuracy: 0.9486
Epoch: [ 0] [  150/  468] time: 36.6989, train_loss: 0.18298885, train_accuracy: 0.9453, test_Accuracy: 0.9485
Epoch: [ 0] [  151/  468] time: 36.9316, train_loss: 0.10354108, train_accuracy: 0.9688, test_Accuracy: 0.9459
Epoch: [ 0] [  152/  468] time: 37.1678, train_loss: 0.07551529, train_accuracy: 0.9844, test_Accuracy: 0.9443
Epoch: [ 0] [  153/  468] time: 37.4011, train_loss: 0.22871764, train_accuracy: 0.9375, test_Accuracy: 0.9435
Epoch: [ 0] [  154/  468] time: 37.6373, train_loss: 0.21388406, train_accuracy: 0.9453, test_Accuracy: 0.9448
Epoch: [ 0] [  155/  468] time: 37.8731, train_loss: 0.19770406, train_accuracy: 0.9453, test_Accuracy: 0.9461
Epoch: [ 0] [  156/  468] time: 38.1038, train_loss: 0.19024861, train_accuracy: 0.9766, test_Accuracy: 0.9445
E

Epoch: [ 0] [  222/  468] time: 53.8447, train_loss: 0.12541947, train_accuracy: 0.9531, test_Accuracy: 0.9573
Epoch: [ 0] [  223/  468] time: 54.0786, train_loss: 0.08777483, train_accuracy: 0.9766, test_Accuracy: 0.9544
Epoch: [ 0] [  224/  468] time: 54.3359, train_loss: 0.12858374, train_accuracy: 0.9453, test_Accuracy: 0.9528
Epoch: [ 0] [  225/  468] time: 54.5714, train_loss: 0.11116992, train_accuracy: 0.9609, test_Accuracy: 0.9519
Epoch: [ 0] [  226/  468] time: 54.8150, train_loss: 0.10917427, train_accuracy: 0.9766, test_Accuracy: 0.9520
Epoch: [ 0] [  227/  468] time: 55.0482, train_loss: 0.11367516, train_accuracy: 0.9531, test_Accuracy: 0.9547
Epoch: [ 0] [  228/  468] time: 55.2819, train_loss: 0.19818626, train_accuracy: 0.9375, test_Accuracy: 0.9585
Epoch: [ 0] [  229/  468] time: 55.5198, train_loss: 0.10250784, train_accuracy: 0.9766, test_Accuracy: 0.9594
Epoch: [ 0] [  230/  468] time: 55.7544, train_loss: 0.11803742, train_accuracy: 0.9688, test_Accuracy: 0.9580
E

Epoch: [ 0] [  296/  468] time: 70.7841, train_loss: 0.07806879, train_accuracy: 0.9688, test_Accuracy: 0.9528
Epoch: [ 0] [  297/  468] time: 71.0086, train_loss: 0.06223743, train_accuracy: 0.9844, test_Accuracy: 0.9572
Epoch: [ 0] [  298/  468] time: 71.2495, train_loss: 0.11219284, train_accuracy: 0.9609, test_Accuracy: 0.9594
Epoch: [ 0] [  299/  468] time: 71.4732, train_loss: 0.13786136, train_accuracy: 0.9688, test_Accuracy: 0.9566
Epoch: [ 0] [  300/  468] time: 71.6893, train_loss: 0.06600509, train_accuracy: 0.9766, test_Accuracy: 0.9539
Epoch: [ 0] [  301/  468] time: 71.9155, train_loss: 0.11150765, train_accuracy: 0.9609, test_Accuracy: 0.9541
Epoch: [ 0] [  302/  468] time: 72.1121, train_loss: 0.24005030, train_accuracy: 0.9453, test_Accuracy: 0.9591
Epoch: [ 0] [  303/  468] time: 72.3334, train_loss: 0.12811598, train_accuracy: 0.9688, test_Accuracy: 0.9639
Epoch: [ 0] [  304/  468] time: 72.5693, train_loss: 0.12190229, train_accuracy: 0.9531, test_Accuracy: 0.9657
E

Epoch: [ 0] [  370/  468] time: 87.4630, train_loss: 0.13178560, train_accuracy: 0.9688, test_Accuracy: 0.9630
Epoch: [ 0] [  371/  468] time: 87.6978, train_loss: 0.10468665, train_accuracy: 0.9688, test_Accuracy: 0.9648
Epoch: [ 0] [  372/  468] time: 87.9246, train_loss: 0.06521379, train_accuracy: 0.9766, test_Accuracy: 0.9646
Epoch: [ 0] [  373/  468] time: 88.1357, train_loss: 0.09734850, train_accuracy: 0.9609, test_Accuracy: 0.9630
Epoch: [ 0] [  374/  468] time: 88.3586, train_loss: 0.11664182, train_accuracy: 0.9609, test_Accuracy: 0.9625
Epoch: [ 0] [  375/  468] time: 88.5903, train_loss: 0.07178049, train_accuracy: 0.9688, test_Accuracy: 0.9631
Epoch: [ 0] [  376/  468] time: 88.8096, train_loss: 0.08798672, train_accuracy: 0.9688, test_Accuracy: 0.9630
Epoch: [ 0] [  377/  468] time: 89.0386, train_loss: 0.09149094, train_accuracy: 0.9844, test_Accuracy: 0.9628
Epoch: [ 0] [  378/  468] time: 89.2629, train_loss: 0.11569055, train_accuracy: 0.9688, test_Accuracy: 0.9635
E

Epoch: [ 0] [  444/  468] time: 104.2324, train_loss: 0.03579625, train_accuracy: 0.9922, test_Accuracy: 0.9640
Epoch: [ 0] [  445/  468] time: 104.4489, train_loss: 0.07723364, train_accuracy: 0.9922, test_Accuracy: 0.9655
Epoch: [ 0] [  446/  468] time: 104.6794, train_loss: 0.06775929, train_accuracy: 0.9688, test_Accuracy: 0.9666
Epoch: [ 0] [  447/  468] time: 104.8951, train_loss: 0.05428031, train_accuracy: 0.9844, test_Accuracy: 0.9664
Epoch: [ 0] [  448/  468] time: 105.1164, train_loss: 0.05989979, train_accuracy: 0.9844, test_Accuracy: 0.9690
Epoch: [ 0] [  449/  468] time: 105.3376, train_loss: 0.02638658, train_accuracy: 1.0000, test_Accuracy: 0.9685
Epoch: [ 0] [  450/  468] time: 105.5576, train_loss: 0.05477978, train_accuracy: 0.9766, test_Accuracy: 0.9678
Epoch: [ 0] [  451/  468] time: 105.7808, train_loss: 0.04640549, train_accuracy: 0.9766, test_Accuracy: 0.9677
Epoch: [ 0] [  452/  468] time: 106.0129, train_loss: 0.11076009, train_accuracy: 0.9766, test_Accuracy: